In [3]:
from cosmicds.app import Application
from glue.core.state_objects import State

def test_instantiate():
    app = Application()
    return issubclass(type(app.state), State)

In [ ]:
def ipykernel_filename():
    import ipykernel
    return ipykernel.connect.get_connection_file()

In [ ]:
def stuff(): 
    import os
    import re
    import json
    import requests
    import ipykernel
    from pywwt.jupyter_relay import _compute_notebook_server_base_url, _list_running_servers_jl3
    kernel_id = re.search(
        "kernel-(.*).json", ipykernel.connect.get_connection_file()
    ).group(1)
    running_server_info = list(_list_running_servers_jl3())
    for s in running_server_info:
        # We need an API token that in most cases is provided in the runtime
        # JSON files. In (recent versions of?) the JupyterHub single-user
        # server, it seems that the token is instead obtained from an
        # environment variable. Cf.
        # https://github.com/jupyterhub/jupyterhub/blob/master/jupyterhub/singleuser/mixins.py
        token = s.get("token", "")
        if not token:
            token = os.environ.get("JUPYTERHUB_API_TOKEN", "")
        if not token:
            token = os.environ.get("JPY_API_TOKEN", "")  # deprecated as of 0.7.2

        # Request/response paranoia due to "fun" figuring out how to fix the
        # JupyterHub single-user problem - the API call would fail due to auth
        # issues and break pywwt, even though there was only one running server
        # so we actually didn't even need the API call. In case something breaks
        # in the future, add a fallback mode.
        try:
            response = requests.get(
                requests.compat.urljoin(s["url"], "api/sessions"),
                params={"token": token},
            )

            for n in json.loads(response.text):
                print(n)
                if n["kernel"]["id"] == kernel_id:
                    return s["base_url"]  # Found it!
        except Exception:
            pass

    # If we got here, we might have auth issues with the api/sessions request.
    # If there's only one server, just give it a try.
    if len(running_server_info) == 1:
        print(running_server_info[0])
        return running_server_info[0]["base_url"]

In [ ]:
def kernel_id():
    import re
    import ipykernel
    kernel_id = re.search(
        "kernel-(.*).json", ipykernel.connect.get_connection_file()
    ).group(1)
    return kernel_id

In [ ]:
def server_names():
    from pywwt.jupyter_relay import _list_running_servers_jl3
    servers = list(_list_running_servers_jl3())
    return servers

In [ ]:
def server_url():
    from pywwt.jupyter_relay import _compute_notebook_server_base_url
    return _compute_notebook_server_base_url()